<center>
<img src="https://www.themoviedb.org/assets/2/v4/logos/v2/blue_long_2-9665a76b1ae401a510ec1e0ca40ddcb3b0cfe45f1d51b77a308fea0845885648.svg" alt="TMDB Logo" class="center" height="50">

# TMDB Project

</center>

## Objective:

The objective of this project is to create a MYSQL database on Movies from a subset of IMDB's publicly available dataset. This database will be explored and analyzed with the end goal of understanding what are the qualities and characteristics that make a movie successful. We will then use these key findings to provide stakeholders with data driven recommendations on how to make a successful movie.

- Part 1: Download several files from IMDB’s movie data set and filter out the subset of movies requested by the stakeholder.
- Part 2: Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.
- Part 3: Construct and export a MySQL database using your data.
- Part 4: Apply hypothesis testing to explore what makes a movie successful.
- Part 5: Produce a Linear Regression model to predict movie performance.

### Data Contributions

This data has been downloaded from The Movie Database (TMDB) which is a community built movie and TV database. 

Link to TMDB Database: [Click Here](https://datasets.imdbws.com/)

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

### Upload Data

In [2]:
#create variables for the datasets
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

### **Title Basics Dictionary**

**Column** | **Description** 
--- | --- 
titleId (string) | a tconst, an alphanumeric unique identifier of the title
tconst (string) | alphanumeric unique identifier of the title
titleType (string) | the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
primaryTitle (string) | the more popular title / the title used by the filmmakers on promotional materials at the point of release
originalTitle (string) | original title, in the original language
isAdult (boolean) | 0: non-adult title; 1: adult title
startYear (YYYY) | represents the release year of a title. In the case of TV Series, it is the series start year
endYear (YYYY) | TV Series end year. ‘\N’ for all other title types
runtimeMinutes | primary runtime of the title, in minutes
genres (string array) | includes up to three genres associated with the title

In [3]:
basics_ = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [4]:
basics_df = basics_.copy()
basics_df.head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"


### **Title Akas Dictionary**

**Column** | **Description** 
--- | --- 
titleId (string) | a tconst, an alphanumeric unique identifier of the title
ordering (integer) | a number to uniquely identify rows for a given titleId
title (string) | the localized title
region (string) | the region for this version of the title
language (string) |the language of the title
types (array) | Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
attributes (array) | Additional terms to describe this alternative title, not enumerated
isOriginalTitle (boolean) | 0: not original title; 1: original title

In [5]:
akas_ = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [6]:
akas_df = akas_.copy()
akas_df.head(2)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0


### **Title Ratings Dictionary**

**Column** | **Description** 
--- | --- 
tconst (string) | alphanumeric unique identifier of the title
averageRating | weighted average of all the individual user ratings
numVotes | number of votes the title has received

In [7]:
ratings_ = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [8]:
ratings_df = ratings_.copy()
ratings_df.head(2)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1969
1,tt0000002,5.8,263


## Inpecting/Cleaning Data

Title Akas Dataset

In [9]:
#verifying how many rows/columns, datatypes, missing items and duplicate rows
print(akas_df.info())
print(('-'*30))
print(f'There are {akas_df.duplicated().sum()} duplicate rows.')
print(('-'*30))
print(f'There are {akas_df.isna().sum().sum()} missing values.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35843078 entries, 0 to 35843077
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB
None
------------------------------
There are 0 duplicate rows.
------------------------------
There are 116 missing values.


In [10]:
#replacing all |N values with np.nan
akas_df.replace({'\\N':np.nan}, inplace=True)

In [11]:
#filtering for only US region
akas_df = akas_df[akas_df['region'] == 'US']

In [12]:
#verifying to see if filtered correctly
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [13]:
#verifying final data
akas_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1435435 entries, 5 to 35842822
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1435435 non-null  object
 1   ordering         1435435 non-null  int64 
 2   title            1435435 non-null  object
 3   region           1435435 non-null  object
 4   language         3914 non-null     object
 5   types            978667 non-null   object
 6   attributes       46540 non-null    object
 7   isOriginalTitle  1434090 non-null  object
dtypes: int64(1), object(7)
memory usage: 98.6+ MB


Title Basics Dataset

In [14]:
#verifying how many rows/columns, datatypes, missing items and duplicate rows
print(basics_df.info())
print(('-'*30))
print(f'There are {basics_df.duplicated().sum()} duplicate rows.')
print(('-'*30))
print(f'There are {basics_df.isna().sum().sum()} missing values.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9837728 entries, 0 to 9837727
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 675.5+ MB
None
------------------------------
There are 0 duplicate rows.
------------------------------
There are 37 missing values.


In [15]:
#replacing all |N values with np.nan
basics_df.replace({'\\N':np.nan}, inplace=True)

In [16]:
#drops rows from just a specified column with null values
basics_df.dropna(subset=['runtimeMinutes','genres'], inplace=True)

In [17]:
#convert datatype from object to float
basics_df['startYear'] = basics_df['startYear'].astype(float)

In [18]:
# filter out only movies
basics_df = basics_df.loc[basics_df['titleType'] == 'movie']
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382047 entries, 8 to 9837678
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          382047 non-null  object 
 1   titleType       382047 non-null  object 
 2   primaryTitle    382047 non-null  object 
 3   originalTitle   382047 non-null  object 
 4   isAdult         382047 non-null  object 
 5   startYear       375606 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  382047 non-null  object 
 8   genres          382047 non-null  object 
dtypes: float64(1), object(8)
memory usage: 29.1+ MB


In [19]:
#filtering years
basics_df = basics_df[(basics_df['startYear']>=2000)&(basics_df['startYear']<2022)]
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021.0,NaN,94,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [20]:
# Exclude movies that are included in the documentary category.
is_documentary = basics_df['genres'].str.contains('documentary',case=False)
basics_df = basics_df[~is_documentary]

In [21]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics_df['tconst'].isin(akas_df['titleId'])
keepers

34803       True
61116       True
67669       True
86801       True
93938       True
           ...  
9837401     True
9837410     True
9837449    False
9837494     True
9837578    False
Name: tconst, Length: 138446, dtype: bool

In [22]:
basics_df = basics_df[keepers]

In [23]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [24]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81710 entries, 34803 to 9837494
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81710 non-null  object 
 1   titleType       81710 non-null  object 
 2   primaryTitle    81710 non-null  object 
 3   originalTitle   81710 non-null  object 
 4   isAdult         81710 non-null  object 
 5   startYear       81710 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  81710 non-null  object 
 8   genres          81710 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.2+ MB


Ratings Database

In [25]:
#verifying how many rows/columns, datatypes, missing items and duplicate rows
print(ratings_df.info())
print(('-'*30))
print(f'There are {ratings_df.duplicated().sum()} duplicate rows.')
print(('-'*30))
print(f'There are {ratings_df.isna().sum().sum()} missing values.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308743 entries, 0 to 1308742
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1308743 non-null  object 
 1   averageRating  1308743 non-null  float64
 2   numVotes       1308743 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.0+ MB
None
------------------------------
There are 0 duplicate rows.
------------------------------
There are 0 missing values.


In [26]:
#replacing all |N values with np.nan
ratings_df.replace({'\\N':np.nan}, inplace=True)

In [27]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers2 =ratings_df['tconst'].isin(akas_df['titleId'])
keepers2

0           True
1           True
2          False
3          False
4           True
           ...  
1308738    False
1308739    False
1308740    False
1308741    False
1308742    False
Name: tconst, Length: 1308743, dtype: bool

In [28]:
ratings_df = ratings_df[keepers2]

In [29]:
ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1969
1,tt0000002,5.8,263
4,tt0000005,6.2,2612
5,tt0000006,5.1,181
6,tt0000007,5.4,818


In [30]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497874 entries, 0 to 1308718
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         497874 non-null  object 
 1   averageRating  497874 non-null  float64
 2   numVotes       497874 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.2+ MB


### Saving Datasets

In [31]:
# creating a new folder with os called Data - If output is just empty brackets [], it worked.
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['.ipynb_checkpoints',
 'final_movies_2000.csv.gz',
 'final_movies_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [32]:
## Save current dataframes to file.
basics_df.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [33]:
# replace original dataframe variable by reading new saved file and confirming it's correct
basics_df = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [34]:
#repeating above steps for akas dataset
akas_df.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
akas_df = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [35]:
#repeating above steps for ratings dataset
ratings_df.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)
ratings_df = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1969
1,tt0000002,5.8,263
2,tt0000005,6.2,2612
3,tt0000006,5.1,181
4,tt0000007,5.4,818
